In [153]:
import requests
import re
import json
import logging
from bs4 import BeautifulSoup as bs

with open('/Users/daeyeop/Work/KBS Drama project/Data/drama_edited.json', 'r', encoding='utf-8') as f:
    drama = json.load(f)
    drama = set(drama)

with open('/Users/daeyeop/Work/KBS Drama project/Data/actor_edited.json', 'r', encoding='utf-8') as f:
    actors = json.load(f)
    actors = set(actors)

base_url = 'https://search.naver.com/search.naver'
logging.basicConfig(filename='Drama.log', level=10, format='%(asctime)s:%(message)s')

def get_soup(url):
    html = requests.get(url)
    assert html.status_code == 200, 'Response needs to be 200, but {}'.format(html)
    soup = bs(html.content, 'html.parser')
    
    return soup


def get_cast_url(drama_url):
    soup = get_soup(drama_url)
    url_list = soup.find('ul',{'class':'tab_list'})
    for i in url_list.find_all('a'):
        try:
            url = i.attrs['href']
        except:
            logging.info('drama_url : {}, no attrs, in {}'.format(i, get_cast_url.__cast__))
            return None
        else:
            if 'cast%' in url:
                cast_url = base_url+url
    return cast_url


def isdrama(url):
    soup = get_soup(url)
    try:
        genre = soup.find('div', {'class':'sub_title'}).next.next.text
    except:
        return False
    if genre == '드라마':
        return True
    return False


def get_drama_urls(actor_url):
    
    pre_len_drama_set = len(drama)
    drama_url_list = []
    soup = get_soup(actor_url)
    result = soup.find('div',{'class':'workact_dsc'})
    try:
        drama_data = result.text.split('oPagingListUi76.displayResult(')[1].split('}]')[0]
    except:
        logging.info('actor_url : {}, no displayResult, in {}'.format(actor_url, get_drama_urls.__name__))
        return None
    for i in drama_data.split('\"onair_url\": \"'):
        if i.startswith('https://search.naver.com/search'):
            drama_url = i.split("\"")[0]
            if isdrama(drama_url) and not drama_url in drama:
                drama_url_list.append(drama_url) 
                drama.add(drama_url)
                print_title(drama_url)
    
    if len(drama_url_list) == 0:
        return None
    if pre_len_drama_set == len(drama):
        return None

    return drama_url_list


def get_actor_urls(cast_url):
    pre_len_actors_set = len(actors)
    actor_url_list = []
    people_search = 'https://people.search.naver.com/search.naver?where=nexearch&sm=tab_ppn&query={query}&os={os}&ie=utf8&key=PeopleService'
    soup = get_soup(cast_url)
    
    url_list = soup.find('div',{'class':"list_image_info"})
    url_list_parts = url_list.find_all('span',{'class':'sub_text type_ell_2 _html_ellipsis'})
    if (url_list_parts) == 0:
        logging.info('cast_url : {}, len is 0, in {}'.format(cast_url, get_actor_urls.__name__))
        return None
    for i in url_list_parts:
        try:
            url = i.a.attrs['href']
            url = url.split('&')
            for part in url:
                if part.startswith('os'):
                    os = part.split('=')[-1]
                elif part.startswith('query'):
                    query = part.split('=')[-1]
        except Exception as e:
            logging.info('actor_url : {}, error is {} in {}'.format(i, e, get_actor_urls.__name__))
            
        else:
            actor_url = people_search.format(query=query, os=os)
            
            if actor_url not in actors:
                actors.add(actor_url)
                actor_url_list.append(actor_url)
                print_title(actor_url)
        
    if len(actor_url_list) == 0:
        return None
    if pre_len_actors_set == len(actors):
        return None
    
    return actor_url_list

def save(google_drive=True, local_drive=True):
    
    drama_file = 'drama_edited_2.json'
    actor_file = 'actor_edited_2.json'
    
    if google_drive:
        with open('/Users/daeyeop/Google Drive File Stream/My drive/코어닷_드라마/{}'.format(drama_file), 'w', encoding='utf-8') as f:
            json.dump(list(drama), f, indent='\t')
        with open('/Users/daeyeop/Google Drive File Stream/My drive/코어닷_드라마/{}'.format(actor_file), 'w', encoding='utf-8') as f:
            json.dump(list(actors), f, indent='\t')
            
        print('drama : {}, actors : {} are saved in google_drive'.format(len(drama), len(actors)))
    
    if local_drive:
        with open('/Users/daeyeop/Work/KBS Drama project/Data/{}'.format(drama_file) , 'w', encoding='utf-8') as f:
            json.dump(list(drama), f, indent='\t')
        with open('/Users/daeyeop/Work/KBS Drama project/Data/{}'.format(actor_file), 'w', encoding='utf-8') as f:
            json.dump(list(actors), f, indent='\t')

    print('drama : {}, actors : {} are saved in local_drive'.format(len(drama), len(actors)))

def crawler(actor_url):
    drama_url_list = get_drama_urls(actor_url)  ### drama.add(drama_urls)
    if drama_url_list is None:
        return None
    for drama_url in drama_url_list:
        cast_url = get_cast_url(drama_url)
        actor_url_list = get_actor_urls(cast_url) ### actors.add(actor_url)
        if actor_url_list is None:
            return None
        for actor_url in actor_url_list:
            crawler(actor_url)
            save()
            
def print_title(url):
    soup = get_soup(url)
    print(soup.title.text)
             

In [156]:
new_act = 'https://people.search.naver.com/search.naver?where=nexearch&query=신성록&sm=tab_etc&ie=utf8&key=PeopleService&os=115385'
for i in actors[-10:]:
    crawler(i)